## Import libraries

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load the dataset

In [2]:
train_data = np.load('dataset/Primary categories - Train.npz')
val_data = np.load('dataset/Primary categories - Validation.npz')
test_data = np.load('dataset/Primary categories - Test.npz')

In [3]:
images_train, labels_train = train_data['images'], train_data['labels']

images_val, labels_val = val_data['images'], val_data['labels']

images_test, labels_test = test_data['images'], test_data['labels']

We have 3 labels here:  
0 for Glasses/Sunglasses  
1 for Trousers and Jeans  
2 for Shoes

In [5]:
images_train.shape

(12963, 120, 90, 3)

In [6]:
np.unique(labels_train)

array([0, 1, 2])

In [7]:
labels_train.shape

(12963,)

In [8]:
images_val.shape

(1620, 120, 90, 3)

In [9]:
images_test.shape

(1620, 120, 90, 3)

In [4]:
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [11]:
images_train.shape

(12963, 120, 90, 3)

## Create the model

### Hyperparameters tuning

In [12]:
# define some hyperparameters
BATCH_SIZE = 64
EPOCHS = 15

In [13]:
# for hyperparameters tuning
HP_KERNEL_SIZE = hp.HParam('kernel size', hp.Discrete([3,5,7]))
HP_KERNEL_NUM = hp.HParam('kernel num', hp.Discrete([32,64,96,128]))

METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 1 (Primary Classification)/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_KERNEL_SIZE, HP_KERNEL_NUM],
        metrics = [hp.Metric(METRIC, display_name = 'accuracy')]
    )

### Model architecture

In [14]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], hparams[HP_KERNEL_SIZE], activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_KERNEL_NUM], 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(3)
    ])

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)

    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 1 (Primary Classification)\\fit\\" + f"run-{session_num}"
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Glasses/Sunglasses','Trousers/Jeans','Shoes'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    # Callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(
        patience = 2,
        restore_best_weights = True
    )
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir, histogram_freq = 1, profile_batch = 0)
    
    model.fit(images_train, 
              labels_train, 
              epochs = EPOCHS, 
              batch_size = BATCH_SIZE, 
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1
             )
    
    _, accuracy = model.evaluate(images_val,labels_val)
    
    model.save(r'saved_models\Model 1 (Primary Classification)\Run-{}'.format(session_num))
    
    return accuracy

## Run the model!

In [15]:
def run(log_dir,hparams,session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [16]:
session_num = 1
# all hparams combination
for kernel_size in HP_KERNEL_SIZE.domain.values:
    for kernel_num in HP_KERNEL_NUM.domain.values:
        hparams = {
            HP_KERNEL_SIZE : kernel_size,
            HP_KERNEL_NUM : kernel_num
        }
        run_name = f"run-{session_num}"
        print('--- Starting Trial:',run_name)
        print({h.name: hparams[h] for h in hparams})
        run("Logs/Model 1 (Primary Classification)/hparam_tuning/" + run_name, hparams, session_num)
        
        session_num += 1

--- Starting Trial: run-1
{'kernel size': 3, 'kernel num': 32}
Epoch 1/15
203/203 [==============================] - 65s 321ms/step - loss: 0.0887 - accuracy: 0.9725 - val_loss: 0.0139 - val_accuracy: 0.9951
Epoch 2/15
203/203 [==============================] - 64s 313ms/step - loss: 0.0154 - accuracy: 0.9981 - val_loss: 0.0033 - val_accuracy: 0.9994
Epoch 3/15
203/203 [==============================] - 64s 315ms/step - loss: 0.0117 - accuracy: 0.9981 - val_loss: 0.0039 - val_accuracy: 0.9988
Epoch 4/15
203/203 [==============================] - 68s 333ms/step - loss: 0.0061 - accuracy: 0.9993 - val_loss: 0.0032 - val_accuracy: 0.9988
Epoch 5/15
203/203 [==============================] - 64s 316ms/step - loss: 0.0058 - accuracy: 0.9990 - val_loss: 0.0145 - val_accuracy: 0.9957
Epoch 6/15
51/51 [==============================] - 2s 45ms/step - loss: 0.0032 - accuracy: 0.9988
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatical